In [ ]:
!pip install ts2vg

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from ts2vg import *
from nltk import ngrams

In [ ]:
shop_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/release_10_23_2020.csv')
shop_df.head()

In [ ]:
print(shop_df.event_type.value_counts())
print(shop_df.product_action.value_counts())

In [ ]:
print(shop_df['product_action'].unique())
shop_df.drop(columns=['event_type', 'product_skus_hash', 'hashed_url'], axis=1, inplace=True)
shop_df.head()

In [ ]:
shop_df['product_action'].replace(np.nan,1, inplace=True)
symbols = {
    'detail': 2,
    'add': 3,
    'remove': 4,
    'purchase': 5,
    'click': 6
}
shop_df['product_action'].replace(symbols, inplace=True)

In [ ]:
shop_df = shop_df.groupby(shop_df.session_id_hash).agg(list).reset_index()
print(shop_df.shape)
shop_df.head()

In [ ]:
# # Truncating till 8
shop_df['conversion_class'] = shop_df['product_action'].apply(lambda x: 'C' if 5 in x else 'NC')
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x[0:x.index(5)] if 5 in x else x)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) >= 8 else np.nan)
shop_df.dropna(inplace=True)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) <= 155 else np.nan)
shop_df.dropna(inplace=True)
print(shop_df.shape)
print(shop_df.conversion_class.value_counts())
shop_df.head()  

In [ ]:
def reduced_time(time):
    index = 0
    for sec in time:
        if index<len(time)-1:                           # do -2 when using all events
            time[index] = abs(time[index+1] - time[index])
        index+=1
    time.pop()

    # final_val = time.pop()
    # final_val = abs(final_val - avg)
    # time.append(final_val)
    
    total_time = sum(time)
    index = 0
    for sec in time:
        time[index] = sec/total_time
        index+=1
    
    return time

shop_preprocessed_df = shop_df.copy()    
shop_preprocessed_df['reduced_time'] = shop_preprocessed_df['server_timestamp_epoch_ms'].apply(lambda x: reduced_time(x))

In [ ]:
shop_preprocessed_df.reset_index(inplace=True)
shop_preprocessed_df.drop(['server_timestamp_epoch_ms', 'index'], axis=1, inplace=True)
shop_preprocessed_df.head()

In [ ]:
def generate_edges(time_list):
    vg = HorizontalVG()
    vg.build(time_list)
    edges = vg.edges
    return edges

shop_preprocessed_df['HVGms_edges'] = shop_preprocessed_df['reduced_time'].apply(lambda x: generate_edges(x))
shop_preprocessed_df.head()

In [ ]:
def pattern_hvg_4(edges):  
    i = 0
    j = edges[len(edges)-1][1]
    pattern = list()
    while i<=int(j/2):
        edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+3]
        edges_tb_used = sorted(edges_tb_used, key=lambda x: (x[0], x[1]))
        # print(i)
        if len(edges_tb_used) == 5:
            if [(i, i+1), (i, i+3), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                pattern.append('F4')
            if [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('E4')
        
        elif len(edges_tb_used) == 4:
            if [(i, i+1), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                pattern.append('D4')
            if [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('C4')
            if [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('B4')
            
        elif len(edges_tb_used) == 3:
            pattern.append('A4')
        
        i+=1
                
    return pattern

In [ ]:
def pattern_hvg_5(edges):
    i = 0
    j = edges[len(edges)-1][1]
    pattern = list()
    # print("=====")
    # print(edges)
    while i<=int(j/2):
        edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+4]
        edges_tb_used = sorted(edges_tb_used, key=lambda x: (x[0], x[1]))

        if len(edges_tb_used) == 7:
            if [(i, i+1), (i, i+2), (i, i+4), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('V5')
            elif [(i, i+1), (i, i+3), (i, i+4), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('U5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+3), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('T5')
            elif [(i, i+1), (i, i+2), (i, i+3), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('S5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('R5')
                
        elif len(edges_tb_used) == 6:
            if [(i, i+1), (i+1, i+2), (i+1, i+3), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('Q5')
            elif [(i, i+1), (i, i+3), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('P5')
            elif [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('O5')
            elif [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('N5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('M5')
            elif [(i, i+1), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('L5')
            elif [(i, i+1), (i, i+2), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('K5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('J5')
            elif [(i, i+1), (i, i+3), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('I5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('H5')    
            
        elif len(edges_tb_used) == 5:
            if [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('G5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('F5')
            elif [(i, i+1), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('E5')
            elif [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('D5')
            elif [(i, i+1), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('C5')
            elif [(i, i+1), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('B5')
            
        elif len(edges_tb_used) == 4:
            pattern.append('A5')
        # print(pattern)    
        i+=1
    return pattern

In [ ]:
shop_preprocessed_df['pattern_hvg_4_nodes'] = shop_preprocessed_df['HVGms_edges'].apply(pattern_hvg_4)
shop_preprocessed_df['pattern_hvg_5_nodes'] = shop_preprocessed_df['HVGms_edges'].apply(pattern_hvg_5)
shop_preprocessed_df.head()

In [ ]:
shop_preprocessed_df['unigram'] = shop_preprocessed_df['product_action'].apply(lambda x: list(ngrams(x, 1)))
shop_preprocessed_df['bigram'] = shop_preprocessed_df['product_action'].apply(lambda x: list(ngrams(x, 2)))
shop_preprocessed_df['trigram'] = shop_preprocessed_df['product_action'].apply(lambda x: list(ngrams(x, 3)))
shop_preprocessed_df.head()

In [ ]:
print(shop_preprocessed_df.unigram)
print(shop_preprocessed_df.bigram)
print(shop_preprocessed_df.trigram)

In [ ]:
shop_preprocessed_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/shopper_data_feature_extracted.csv', index=False)